DESCOBERTAS:

- VIP importa
    * ~50% dos Não-VIPs são transportados
    * ~38% dos VIPs são transportados
    * Terra implica Não-VIP
    * Mas está baixo na lista de prioridade para ser transportado
        * Europa -> ~65% Transportados
        * Earth -> ~41% Transportados -> é o HomePlanet com maior número de pessoas (3566) e todas são NÃO-VIPs
        * Mars -> ~53% Transportados
    *~2.4% dos passageiros são VIPs
    * idade <= 17 eh nao VIP
- Usando cabines para os planetas
    * A B C T -> Europa
    * G -> Earth
    * F -> 60% Earth, 40% Mars
    * E -> 45% Earth, 40% Mars, 15% Europa
    * D -> 60% Mars, 40% Europa
- Cryosleep True ou age <= 12 implica nenhum gasto OK
- Qualquer gasto implica em Cryosleep False OK
- Msm grupo -> Msm Planeta  OK
- Msm sobrenome -> Msm Planeta  OK
- Cada HomePlanet tem uma Destination preferida
    * Earth -> PSO
    * Europa -> Cancri
    * Mars -> TRAPPIST
- G e T Sao NAO VIPS
- Destino do T é TRAPPIST
- Todos os T nao tem cryo
- Se a pessoa eh marte e cryosleep, vip falso
- Se a pessoa ta em cryosleep e eh vip, entao EUROPA
### TESTAR
    
    

In [109]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None

In [110]:
df = pd.read_csv("train.csv")
train_df = pd.read_csv("train.csv").dropna()

def setExpensesToZero(index):
    df['RoomService'][index] = 0 
    df['FoodCourt'][index] = 0 
    df['ShoppingMall'][index] = 0 
    df['VRDeck'][index] = 0 


def CryoOr12Expenses(df):
    for i in range(len(df)):
        if (not pd.isna(df['CryoSleep'][i]) and df['CryoSleep'][i] == True) or (not pd.isna(df['Age'][i]) and df['Age'][i] <= 12):
            setExpensesToZero(i)
    return df


#Se a pessoa tem algum gasto, entao coloca cryo como false
def checkForExpenses(i, df):
    if (not pd.isna(df['RoomService'][i]) and df['RoomService'][i] > 0) or (not pd.isna(df['FoodCourt'][i]) and df['FoodCourt'][i] > 0) or (not pd.isna(df['ShoppingMall'][i]) and df['ShoppingMall'][i] > 0) or (not pd.isna(df['VRDeck'][i]) and df['VRDeck'][i] > 0):
        return 0.0
    return 1.0


#formatting data TODO ISSO VEM POR ULTIMO
#df['Transported'] = df['Transported'].map({False: 0, True: 1})
#df['Earth'] = np.select([df['HomePlanet'] == 'Earth'], [1], default = 0)
#df['Europa'] = np.select([df['HomePlanet'] == 'Europa'], [1], default = 0)
#df['Mars'] = np.select([df['HomePlanet'] == 'Mars'], [1], default = 0)
#for i in range (len(df)):
#     if pd.isna(df['CryoSleep'][i]) and (not pd.isna(df['Age'][i]) and df['Age'][i] > 12):
#         df['CryoSleep'][i] = checkForExpenses(i)
#     elif df['CryoSleep'][i] == True:
#         df['CryoSleep'][i] = 1
#     else:
#         df['CryoSleep'][i] = 0
# df['VIP'] = np.select([df['VIP'] == True], [1], default = 0)
# df['TRAP'] = np.select([df['Destination'] == 'TRAPPIST-1e'], [1], default = 0)
# df['PSO'] = np.select([df['Destination'] == 'PSO J318.5-22'], [1], default = 0)
# df['Cancri'] = np.select([df['Destination'] == '55 Cancri e'], [1], default = 0)

In [111]:
def buildCabinColumns(df):
    cabins = []
    cabside = []
    for i in range(len(df)):
        if (pd.isna(df['Cabin'][i])):
            cabins.append(None)
            cabside.append(None)
        else:
            cabins.append(str(df['Cabin'][i][0]))
            cabside.append(str(df['Cabin'][i][-1]))
    df['Cabin0'] = cabins
    df['CabinSide'] = cabside
    return df

df = buildCabinColumns(df)

#test = pd.DataFrame(train_df['Age'])
#test['bins'] = pd.cut(test.Age, [0, 10, 20, 30, 40, 50, 60, 70, 80], include_lowest = True)
#test['bins'].value_counts()
#sns.barplot(x = train_df['Cabin0'], y = train_df['Earth'])
#sns.barplot(x = train_df['Cabin0'], y = train_df['Mars'])
#sns.barplot(x = train_df['Cancri'], y = train_df['VIP'])
#sns.barplot(x = train_df['Cabin0'], y = train_df['Transported'])
#sns.barplot(x = train_df['TRAP'], y = train_df['CryoSleep'])
#sns.barplot(x = train_df['VIP'], y = train_df['Transported'])
#sns.barplot(x = train_df['Europa'], y = train_df['CryoSleep'], hue = train_df['Destination'])
#plt.figure(figsize=(10,6))
#sns.countplot(x = test['bins'], hue=train_df[''])

In [112]:
# This function helps us prove that everybody with the same last name has the same Home Planet.
# The same applies to passenger groups.
def buildHomePlanetDict(column, char, pos, dff):
    '''Function that builds a dictionary that follows a <column, set<planets>> relation'''
    dictionary = {}
    for i in range(len(dff)):
        try:
            current = dff[column][i].split(char)[pos]
            if current not in dictionary:
                dictionary[current] = set()
            if pd.isnull(dff['HomePlanet'][i]) == False:
                dictionary[current].add(dff['HomePlanet'][i])
        except AttributeError:
            continue
    return dictionary

def buildCabSideDict(column, char, pos, dff):
    dictionary = {}
    for i in range(len(dff)):
        try:
            current = dff[column][i].split(char)[pos]
            if current not in dictionary:
                dictionary[current] = set()
            if pd.isnull(dff['CabinSide'][i]) == False:
                dictionary[current].add(dff['CabinSide'][i])
        except AttributeError:
            continue
    return dictionary


groupDict = buildHomePlanetDict('PassengerId', '_', 0, df)
nameDict = buildHomePlanetDict('Name', ' ', 1, df)
cabinDict = {"A": "Europa", "B": "Europa", "C": "Europa", "T": "Europa", "G": "Earth", "F": "Earth", "D": "Mars", "E": "Earth"}
planetDict = {"Earth" : "PSO J318.5-22", "Europa" : "55 Cancri e", "Mars" : "TRAPPIST-1e"} #Preferred destination for each HomePlanet
cabsideDict = {"Earth": "P", "Mars": "P", "Europa": "S"}
groupCabSideDict = buildCabSideDict('PassengerId', '_', 0, df)

def fillCabSideWFavorite(df):
    for i in range(len(df)):
        if (pd.isna(df['CabinSide'][i])):
            df['CabinSide'][i] = cabsideDict[df['HomePlanet'][i]]
    return df
        
   

def getGroup(index, df):
    return df['PassengerId'][index].split('_')[0]

def getSurname(index, df):
    if pd.isna(df['Name'][index]):
        return None
    return df['Name'][index].split(' ')[1]

def getCabin(index):
    if pd.isna(df['Cabin0'][index]):
        return None
    return df['Name'][index].split(' ')[1]

def removeEmptySets(dict):
    deletions = []
    for key in dict.keys():
        if len(dict[key])==0:
            deletions.append(key)
    for key in deletions:
        del dict[key]
    return dict

groupDict = removeEmptySets(groupDict)
nameDict = removeEmptySets(nameDict)
cabsideDict = removeEmptySets(cabsideDict)
def buildGroupsNSurnames(df):
    groups = []
    surnames = []
    for i in range(len(df)):
        groups.append(getGroup(i, df))
        surnames.append(getSurname(i, df))
    df['Groups'] = groups
    df['Surname'] = surnames
    return df

def formatDictionary(dict):
    for el in dict.keys():
        for k in dict[el]:
            dict[el] = k
    return dict

df = buildGroupsNSurnames(df)
df = CryoOr12Expenses(df)


groupDict = formatDictionary(groupDict)
nameDict = formatDictionary(nameDict)
cabsideDict = formatDictionary(cabsideDict)

In [113]:
def fillHomePlanet(df, groupDict, nameDict):
    df['HomePlanet'] = df['HomePlanet'].fillna(df['Groups'].map(groupDict)) 
    df['HomePlanet'] = df['HomePlanet'].fillna(df['Surname'].map(nameDict))
    df['HomePlanet'] = df['HomePlanet'].fillna(df['Cabin0'].map(cabinDict))
    return df

def fillDestWFavorite(df):
    df['Destination'] = df['Destination'].fillna(df['HomePlanet'].map(planetDict))
    return df

df = fillHomePlanet(df, groupDict, nameDict)
df = fillDestWFavorite(df)

def fillCostsWMedian(df):
    df['RoomService'].fillna(df['RoomService'].median(), inplace=True)
    df['Spa'].fillna(df['Spa'].median(), inplace=True)
    df['VRDeck'].fillna(df['VRDeck'].median(), inplace=True)
    df['FoodCourt'].fillna(df['FoodCourt'].median(), inplace=True)
    df['ShoppingMall'].fillna(df['ShoppingMall'].median(), inplace=True)
    return df

df = fillCostsWMedian(df)

def fillWAvgAge(df):
    sumEarth = 0
    indexEarth = 0
    sumEuropa = 0
    indexEuropa = 0
    sumMars = 0
    indexMars= 0

    for i in range(len(df)):
        if not pd.isna(df['HomePlanet'][i]) and not pd.isna(df['Age'][i]):
            if df['HomePlanet'][i] == 'Earth':
                indexEarth += 1
                sumEarth += df['Age'][i]
            elif df['HomePlanet'][i] == 'Europa':
                indexEuropa += 1
                sumEuropa += df['Age'][i]
            elif df['HomePlanet'][i] == 'Mars':
                indexMars += 1
                sumMars += df['Age'][i]

    EarthAgeAvg = sumEarth/indexEarth
    EuropaAgeAvg = sumEuropa/indexEuropa
    MarsAgeAvg = sumMars/indexMars

    for i in range(len(df)):
        if pd.isna(df['Age'][i]):
            if pd.isna(df['HomePlanet'][i]):
                continue
            elif df['HomePlanet'][i] == "Earth":
                df['Age'][i] = EarthAgeAvg
            elif df['HomePlanet'][i] == "Europa":
                df['Age'][i] = EuropaAgeAvg
            else:
                df['Age'][i] = MarsAgeAvg
    return df

df = fillWAvgAge(df)


In [114]:
df['Transported'] = df['Transported'].map({False: 0, True: 1})

def formatHomePlanet(df):   
    df['Earth'] = np.select([df['HomePlanet'] == 'Earth'], [1], default = 0)
    df['Europa'] = np.select([df['HomePlanet'] == 'Europa'], [1], default = 0)
    df['Mars'] = np.select([df['HomePlanet'] == 'Mars'], [1], default = 0)
    return df

df = formatHomePlanet(df)

def buildCryoColumn(df):
    cryocolumn = []
    for i in range (len(df)):
        if pd.isna(df['CryoSleep'][i]) and (not pd.isna(df['Age'][i]) and df['Age'][i] > 12):
            cryocolumn.append(checkForExpenses(i, df))
        elif df['CryoSleep'][i] == True:
            cryocolumn.append(1)
        else:
            cryocolumn.append(0)
    df['CryoSleep'] = cryocolumn
    return df

df = buildCryoColumn(df)

def formatDestination(df):
    df['TRAP'] = np.select([df['Destination'] == 'TRAPPIST-1e'], [1], default = 0)
    df['PSO'] = np.select([df['Destination'] == 'PSO J318.5-22'], [1], default = 0)
    df['Cancri'] = np.select([df['Destination'] == '55 Cancri e'], [1], default = 0)
    return df

df['VIP'] = np.select([df['VIP'] == True], [1], default = 0)
df = formatDestination(df)
df = fillCabSideWFavorite(df) 
df = df.dropna()
df['CabinSide'] = df['CabinSide'].map({'S': 0, 'P': 1})

df = df.drop(columns=['Name', 'Surname', 'Cabin', 'Cabin0', 'HomePlanet', 'Destination', 'Groups', 'PassengerId'])

X = np.array(df.drop(columns=['Transported'], axis=1))
y = np.array(df['Transported'])
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 13, test_size = 0.4)

In [115]:
train_pred = cross_val_predict(RandomForestClassifier(random_state = 1), x_train, y_train, cv = 10, n_jobs=-1)
acuracia_vc = round(accuracy_score(y_train, train_pred)*100, 2)
acuracia_vc

79.34

In [116]:
#Submission block
full_data_model = RandomForestClassifier(random_state = 1)
full_data_model.fit(X,y) #using entire train dataset
test_X = pd.read_csv('test.csv')
# TODO Format test_X

test_X = buildCabinColumns(test_X)
groupDictTestX = buildHomePlanetDict('PassengerId', '_', 0, test_X)
nameDictTestX = buildHomePlanetDict('Name', ' ', 1, test_X)
groupCabSideDictTestX = buildCabSideDict('PassengerId', '_', 0, test_X)


test_X = buildGroupsNSurnames(test_X)
test_X = CryoOr12Expenses(test_X)

groupDictTestX = removeEmptySets(groupDictTestX)
nameDictTestX = removeEmptySets(nameDictTestX)
groupCabSideDictTestX = removeEmptySets(groupCabSideDictTestX)


groupDictTestX = formatDictionary(groupDictTestX)
nameDictTestX = formatDictionary(nameDictTestX)
groupCabSideDictTestX = formatDictionary(groupCabSideDictTestX)


test_X = fillHomePlanet(test_X, groupDictTestX, nameDictTestX)
test_X = fillDestWFavorite(test_X)

test_X = fillCostsWMedian(test_X)

test_X = fillWAvgAge(test_X)

test_X = fillCabSideWFavorite(test_X)

test_X = formatHomePlanet(test_X)

test_X = buildCryoColumn(test_X)

test_X['VIP'] = np.select([test_X['VIP'] == True], [1], default = 0)
test_X = formatDestination(test_X)
test_X['CabinSide'] = test_X['CabinSide'].map({'S': 0, 'P': 1})
passId = test_X['PassengerId']
test_X = test_X.drop(columns=['Name', 'Cabin', 'HomePlanet', 'Destination', 'PassengerId', 'Cabin0', 'Groups', 'Surname'])
test_X.isnull().sum() 
# End TODO
test_preds = full_data_model.predict(test_X)
output = pd.DataFrame({'PassengerId': passId, 'Transported': test_preds})
output['Transported'] = output['Transported'].astype(bool)
output
output.to_csv('submission.csv', index=False)

c:\Users\Rafael\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
